## Investigating which resampling technique will results in balance of fairness acroos metrics of statistical partiy and equalized odds

In [1]:
import pandas as pd

# Load the data
file_path = 'data/df_cleaned.csv'
df = pd.read_csv(file_path)

df.head()

,sex,juv_fel_count,juv_misd_count,juv_other_count,priors_count,two_year_recid,age_cat_25-45,age_cat_Greaterthan45,age_cat_Lessthan25,charge_degree,race
0,1,-0.14965,-0.190564,-0.247266,-0.709995,1,1,0,0,1,1
1,1,-0.14965,-0.190564,1.827564,0.110444,1,0,0,1,1,1
2,1,-0.14965,-0.190564,-0.247266,2.161540,1,1,0,0,1,0
3,0,-0.14965,-0.190564,-0.247266,-0.709995,0,1,0,0,0,0
4,1,-0.14965,-0.190564,-0.247266,-0.709995,0,1,0,0,1,0


In [2]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix, roc_auc_score
from fairness_metrics import statistical_parity, equalized_odds
from metricFrame_vis import analyze_metrics_using_metricFrame

In [3]:
# Initialize a DataFrame to store the results of our analysis for esay comparison

resampling_results_df = pd.DataFrame(columns=[
    'resampling_group',
    'model_accuracy',
    'AUC', 
    'f1', 
    'precision', 
    'recall', 
    'statistical_parity_sex', 
    'equalized_odds_sex',
    'statistical_parity_race',
    'equalized_odds_race',
])

# Function to update the DataFrame with new results
def update_results(resampling_group,
                   model_accuracy, 
                   AUC, 
                   f1, 
                   precision, 
                   recall, 
                   statistical_parity_sex, 
                   equalized_odds_sex, 
                   statistical_parity_race, 
                   equalized_odds_race
                  ):
    new_entry = {
        'resampling_group': resampling_group, 
        'model_accuracy': model_accuracy, 
        'AUC': AUC, 
        'f1': f1, 
        'precision': precision, 
        'recall': recall, 
        'statistical_parity_sex': statistical_parity_sex, 
        'equalized_odds_sex': equalized_odds_sex,
        'statistical_parity_race': statistical_parity_race, 
        'equalized_odds_race': equalized_odds_race, 
    }
    return resampling_results_df.append(new_entry, ignore_index=True)

# Example usage after evaluating a model
# resampling_results_df = update_results(
#     resampling_group='upsampling',
#     model_accuracy=0.85,  
#     AUC=0.90,  
#     f1=0.87, 
#     precision=0.88,  
#     recall=0.86, 
#     statistical_parity_sex =0.89, 
#     equalized_odds_sex =0.99,
#     statistical_parity_race = 0.23, 
#     equalized_odds_race =0.33
# )

## 1. Upsampled_race_sex

In [4]:
# Upsampling RACE feature
group_1 = df[df['race'] == 1]
group_0 = df[df['race'] == 0]


# Function to resample each subgroup to have the same number of samples
def resample_group(df, group_size):
    return df.sample(n=group_size, replace=True, random_state=42)

max_group_size = max(len(group_0[group_0['two_year_recid'] == 1]), len(group_1[group_1['two_year_recid'] == 1]))

# Resample each group to have the same number of samples
group_0_recidivate = resample_group(group_0[group_0['two_year_recid'] == 1], max_group_size)
group_1_recidivate = resample_group(group_1[group_1['two_year_recid'] == 1], max_group_size)

group_0_not_recidivate = resample_group(group_0[group_0['two_year_recid'] == 0], max_group_size)
group_1_not_recidivate = resample_group(group_1[group_1['two_year_recid'] == 0], max_group_size)

# Combine the resampled data back
df_upsampled_race = pd.concat([group_0_recidivate, group_0_not_recidivate, group_1_recidivate, group_1_not_recidivate])

# Shuffle the resampled data
df_upsampled_race = df_upsampled_race.sample(frac=1).reset_index(drop=True)

# Upsampling SEX feature
group_1 = df_upsampled_race[df_upsampled_race['sex'] == 1]
group_0 = df_upsampled_race[df_upsampled_race['sex'] == 0]

# Function to resample each subgroup to have the same number of samples
def resample_group(df, group_size):
    return df.sample(n=group_size, replace=True, random_state=42)

max_group_size = max(len(group_0[group_0['two_year_recid'] == 1]), len(group_1[group_1['two_year_recid'] == 1]))

# Resample each group to have the same number of samples
group_0_recidivate = resample_group(group_0[group_0['two_year_recid'] == 1], max_group_size)
group_1_recidivate = resample_group(group_1[group_1['two_year_recid'] == 1], max_group_size)

group_0_not_recidivate = resample_group(group_0[group_0['two_year_recid'] == 0], max_group_size)
group_1_not_recidivate = resample_group(group_1[group_1['two_year_recid'] == 0], max_group_size)

# Combine the resampled data back
df_upsampled_race_sex_max = pd.concat([group_0_recidivate, group_0_not_recidivate, group_1_recidivate, group_1_not_recidivate])

# Shuffle the resampled data
df_upsampled_race_sex_max = df_upsampled_race_sex_max.sample(frac=1).reset_index(drop=True)

# Data Preparation
X = df_upsampled_race_sex_max.drop(['two_year_recid'], axis=1)
y = df_upsampled_race_sex_max['two_year_recid']

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Remove sex and race from the training data
X_train = X_train.drop(['sex', 'race'], axis=1)
sensitive_feature_race = X_test['race']
sensitive_feature_sex = X_test['sex']
X_test = X_test.drop(['sex', 'race'], axis=1)

# Initialize the Gradient Boosting classifier
gb_classifier = GradientBoostingClassifier(learning_rate=0.01, 
                                           max_depth=3, 
                                           n_estimators=300, 
                                           random_state=42)

# Fit the grid search to the data
gb_classifier.fit(X_train, y_train)

# Evaluate the model on the test set
y_pred_gb = gb_classifier.predict(X_test)

model_accuracy_gb = accuracy_score(y_test, y_pred_gb)
AUC = roc_auc_score(y_test, y_pred_gb)
f1 = f1_score(y_test, y_pred_gb)
precision = precision_score(y_test, y_pred_gb)
recall = recall_score(y_test, y_pred_gb)

statistical_parity_race = statistical_parity(y_test, y_pred_gb, sensitive_feature_race)
equalized_odds_race = equalized_odds(y_test, y_pred_gb, sensitive_feature_race)

statistical_parity_sex = statistical_parity(y_test, y_pred_gb, sensitive_feature_sex)
equalized_odds_sex = equalized_odds(y_test, y_pred_gb, sensitive_feature_sex)

resampling_results_df = update_results(
    resampling_group='Upsampled_Race_Sex',
    model_accuracy=model_accuracy_gb,  
    AUC=AUC,  
    f1=f1, 
    precision=precision,  
    recall=recall, 
    statistical_parity_sex =statistical_parity_sex, 
    equalized_odds_sex =equalized_odds_sex,
    statistical_parity_race = statistical_parity_race, 
    equalized_odds_race = equalized_odds_race
)

/var/folders/h5/r3ldzs6x5bxb71r4t20ydphh0000gn/T/ipykernel_98044/774575340.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return resampling_results_df.append(new_entry, ignore_index=True)


## 2. Downsampled_Race_Sex

In [5]:
# Upsampling RACE feature
group_1 = df[df['race'] == 1]
group_0 = df[df['race'] == 0]


# Function to resample each subgroup to have the same number of samples
def resample_group(df, group_size):
    return df.sample(n=group_size, replace=True, random_state=42)

max_group_size = min(len(group_0[group_0['two_year_recid'] == 1]), len(group_1[group_1['two_year_recid'] == 1]))

# Resample each group to have the same number of samples
group_0_recidivate = resample_group(group_0[group_0['two_year_recid'] == 1], max_group_size)
group_1_recidivate = resample_group(group_1[group_1['two_year_recid'] == 1], max_group_size)

group_0_not_recidivate = resample_group(group_0[group_0['two_year_recid'] == 0], max_group_size)
group_1_not_recidivate = resample_group(group_1[group_1['two_year_recid'] == 0], max_group_size)

# Combine the resampled data back
df_upsampled_race = pd.concat([group_0_recidivate, group_0_not_recidivate, group_1_recidivate, group_1_not_recidivate])

# Shuffle the resampled data
df_upsampled_race = df_upsampled_race.sample(frac=1).reset_index(drop=True)

# Upsampling SEX feature
group_1 = df_upsampled_race[df_upsampled_race['sex'] == 1]
group_0 = df_upsampled_race[df_upsampled_race['sex'] == 0]

# Function to resample each subgroup to have the same number of samples
def resample_group(df, group_size):
    return df.sample(n=group_size, replace=True, random_state=42)

max_group_size = min(len(group_0[group_0['two_year_recid'] == 1]), len(group_1[group_1['two_year_recid'] == 1]))

# Resample each group to have the same number of samples
group_0_recidivate = resample_group(group_0[group_0['two_year_recid'] == 1], max_group_size)
group_1_recidivate = resample_group(group_1[group_1['two_year_recid'] == 1], max_group_size)

group_0_not_recidivate = resample_group(group_0[group_0['two_year_recid'] == 0], max_group_size)
group_1_not_recidivate = resample_group(group_1[group_1['two_year_recid'] == 0], max_group_size)

# Combine the resampled data back
df_upsampled_race_sex_max = pd.concat([group_0_recidivate, group_0_not_recidivate, group_1_recidivate, group_1_not_recidivate])

# Shuffle the resampled data
df_upsampled_race_sex_max = df_upsampled_race_sex_max.sample(frac=1).reset_index(drop=True)

# Data Preparation
X = df_upsampled_race_sex_max.drop(['two_year_recid'], axis=1)
y = df_upsampled_race_sex_max['two_year_recid']

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Remove sex and race from the training data
X_train = X_train.drop(['sex', 'race'], axis=1)
sensitive_feature_race = X_test['race']
sensitive_feature_sex = X_test['sex']
X_test = X_test.drop(['sex', 'race'], axis=1)

# Initialize the Gradient Boosting classifier
gb_classifier = GradientBoostingClassifier(learning_rate=0.01, 
                                           max_depth=3, 
                                           n_estimators=300, 
                                           random_state=42)

# Fit the grid search to the data
gb_classifier.fit(X_train, y_train)

# Evaluate the model on the test set
y_pred_gb = gb_classifier.predict(X_test)

model_accuracy_gb = accuracy_score(y_test, y_pred_gb)
AUC = roc_auc_score(y_test, y_pred_gb)
f1 = f1_score(y_test, y_pred_gb)
precision = precision_score(y_test, y_pred_gb)
recall = recall_score(y_test, y_pred_gb)

statistical_parity_race = statistical_parity(y_test, y_pred_gb, sensitive_feature_race)
equalized_odds_race = equalized_odds(y_test, y_pred_gb, sensitive_feature_race)

statistical_parity_sex = statistical_parity(y_test, y_pred_gb, sensitive_feature_sex)
equalized_odds_sex = equalized_odds(y_test, y_pred_gb, sensitive_feature_sex)

resampling_results_df = update_results(
    resampling_group='Downsampled_Race_Sex',
    model_accuracy=model_accuracy_gb,  
    AUC=AUC,  
    f1=f1, 
    precision=precision,  
    recall=recall, 
    statistical_parity_sex =statistical_parity_sex, 
    equalized_odds_sex =equalized_odds_sex,
    statistical_parity_race = statistical_parity_race, 
    equalized_odds_race = equalized_odds_race
)

/var/folders/h5/r3ldzs6x5bxb71r4t20ydphh0000gn/T/ipykernel_98044/774575340.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return resampling_results_df.append(new_entry, ignore_index=True)


## 3. Upsampling Race Downsampling Sex

In [6]:
# Upsampling_Race_Downsampling_Sex
group_1 = df[df['race'] == 1]
group_0 = df[df['race'] == 0]


# Function to resample each subgroup to have the same number of samples
def resample_group(df, group_size):
    return df.sample(n=group_size, replace=True, random_state=42)

max_group_size = max(len(group_0[group_0['two_year_recid'] == 1]), len(group_1[group_1['two_year_recid'] == 1]))

# Resample each group to have the same number of samples
group_0_recidivate = resample_group(group_0[group_0['two_year_recid'] == 1], max_group_size)
group_1_recidivate = resample_group(group_1[group_1['two_year_recid'] == 1], max_group_size)

group_0_not_recidivate = resample_group(group_0[group_0['two_year_recid'] == 0], max_group_size)
group_1_not_recidivate = resample_group(group_1[group_1['two_year_recid'] == 0], max_group_size)

# Combine the resampled data back
df_upsampled_race = pd.concat([group_0_recidivate, group_0_not_recidivate, group_1_recidivate, group_1_not_recidivate])

# Shuffle the resampled data
df_upsampled_race = df_upsampled_race.sample(frac=1).reset_index(drop=True)

# Upsampling SEX feature
group_1 = df_upsampled_race[df_upsampled_race['sex'] == 1]
group_0 = df_upsampled_race[df_upsampled_race['sex'] == 0]

# Function to resample each subgroup to have the same number of samples
def resample_group(df, group_size):
    return df.sample(n=group_size, replace=True, random_state=42)

max_group_size = min(len(group_0[group_0['two_year_recid'] == 1]), len(group_1[group_1['two_year_recid'] == 1]))

# Resample each group to have the same number of samples
group_0_recidivate = resample_group(group_0[group_0['two_year_recid'] == 1], max_group_size)
group_1_recidivate = resample_group(group_1[group_1['two_year_recid'] == 1], max_group_size)

group_0_not_recidivate = resample_group(group_0[group_0['two_year_recid'] == 0], max_group_size)
group_1_not_recidivate = resample_group(group_1[group_1['two_year_recid'] == 0], max_group_size)

# Combine the resampled data back
df_upsampled_race_sex_max = pd.concat([group_0_recidivate, group_0_not_recidivate, group_1_recidivate, group_1_not_recidivate])

# Shuffle the resampled data
df_upsampled_race_sex_max = df_upsampled_race_sex_max.sample(frac=1).reset_index(drop=True)

# Data Preparation
X = df_upsampled_race_sex_max.drop(['two_year_recid'], axis=1)
y = df_upsampled_race_sex_max['two_year_recid']

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Remove sex and race from the training data
X_train = X_train.drop(['sex', 'race'], axis=1)
sensitive_feature_race = X_test['race']
sensitive_feature_sex = X_test['sex']
X_test = X_test.drop(['sex', 'race'], axis=1)

# Initialize the Gradient Boosting classifier
gb_classifier = GradientBoostingClassifier(learning_rate=0.01, 
                                           max_depth=3, 
                                           n_estimators=300, 
                                           random_state=42)

# Fit the grid search to the data
gb_classifier.fit(X_train, y_train)

# Evaluate the model on the test set
y_pred_gb = gb_classifier.predict(X_test)

model_accuracy_gb = accuracy_score(y_test, y_pred_gb)
AUC = roc_auc_score(y_test, y_pred_gb)
f1 = f1_score(y_test, y_pred_gb)
precision = precision_score(y_test, y_pred_gb)
recall = recall_score(y_test, y_pred_gb)

statistical_parity_race = statistical_parity(y_test, y_pred_gb, sensitive_feature_race)
equalized_odds_race = equalized_odds(y_test, y_pred_gb, sensitive_feature_race)

statistical_parity_sex = statistical_parity(y_test, y_pred_gb, sensitive_feature_sex)
equalized_odds_sex = equalized_odds(y_test, y_pred_gb, sensitive_feature_sex)

In [7]:
resampling_results_df = update_results(
    resampling_group='Upsampling_Race_Downsampling_Sex',
    model_accuracy=model_accuracy_gb,  
    AUC=AUC,  
    f1=f1, 
    precision=precision,  
    recall=recall, 
    statistical_parity_sex =statistical_parity_sex, 
    equalized_odds_sex =equalized_odds_sex,
    statistical_parity_race = statistical_parity_race, 
    equalized_odds_race = equalized_odds_race
)

/var/folders/h5/r3ldzs6x5bxb71r4t20ydphh0000gn/T/ipykernel_98044/774575340.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return resampling_results_df.append(new_entry, ignore_index=True)


In [8]:
resampling_results_df

,resampling_group,model_accuracy,AUC,f1,precision,recall,statistical_parity_sex,equalized_odds_sex,statistical_parity_race,equalized_odds_race
0,Upsampled_Race_Sex,0.668972,0.668935,0.647002,0.692519,0.607100,"{'Statistical Parity Difference': 0.073, 'Stat...","{'Equalized Odds Difference': 0.073, 'Equalize...","{'Statistical Parity Difference': 0.138, 'Stat...","{'Equalized Odds Difference': 0.153, 'Equalize..."
1,Downsampled_Race_Sex,0.672515,0.674180,0.621622,0.747967,0.531792,"{'Statistical Parity Difference': 0.143, 'Stat...","{'Equalized Odds Difference': 0.202, 'Equalize...","{'Statistical Parity Difference': 0.053, 'Stat...","{'Equalized Odds Difference': 0.152, 'Equalize..."
2,Upsampling_Race_Downsampling_Sex,0.649547,0.647168,0.619672,0.630000,0.609677,"{'Statistical Parity Difference': 0.069, 'Stat...","{'Equalized Odds Difference': 0.109, 'Equalize...","{'Statistical Parity Difference': 0.196, 'Stat...","{'Equalized Odds Difference': 0.19, 'Equalized..."


## 4. Downsampling_Race_Upsampling_Sex


In [9]:
# Downsampling_Race_Upsampling_Sex
group_1 = df[df['race'] == 1]
group_0 = df[df['race'] == 0]


# Function to resample each subgroup to have the same number of samples
def resample_group(df, group_size):
    return df.sample(n=group_size, replace=True, random_state=42)

max_group_size = min(len(group_0[group_0['two_year_recid'] == 1]), len(group_1[group_1['two_year_recid'] == 1]))

# Resample each group to have the same number of samples
group_0_recidivate = resample_group(group_0[group_0['two_year_recid'] == 1], max_group_size)
group_1_recidivate = resample_group(group_1[group_1['two_year_recid'] == 1], max_group_size)

group_0_not_recidivate = resample_group(group_0[group_0['two_year_recid'] == 0], max_group_size)
group_1_not_recidivate = resample_group(group_1[group_1['two_year_recid'] == 0], max_group_size)

# Combine the resampled data back
df_upsampled_race = pd.concat([group_0_recidivate, group_0_not_recidivate, group_1_recidivate, group_1_not_recidivate])

# Shuffle the resampled data
df_upsampled_race = df_upsampled_race.sample(frac=1).reset_index(drop=True)

# Upsampling SEX feature
group_1 = df_upsampled_race[df_upsampled_race['sex'] == 1]
group_0 = df_upsampled_race[df_upsampled_race['sex'] == 0]

# Function to resample each subgroup to have the same number of samples
def resample_group(df, group_size):
    return df.sample(n=group_size, replace=True, random_state=42)

max_group_size = max(len(group_0[group_0['two_year_recid'] == 1]), len(group_1[group_1['two_year_recid'] == 1]))

# Resample each group to have the same number of samples
group_0_recidivate = resample_group(group_0[group_0['two_year_recid'] == 1], max_group_size)
group_1_recidivate = resample_group(group_1[group_1['two_year_recid'] == 1], max_group_size)

group_0_not_recidivate = resample_group(group_0[group_0['two_year_recid'] == 0], max_group_size)
group_1_not_recidivate = resample_group(group_1[group_1['two_year_recid'] == 0], max_group_size)

# Combine the resampled data back
df_upsampled_race_sex_max = pd.concat([group_0_recidivate, group_0_not_recidivate, group_1_recidivate, group_1_not_recidivate])

# Shuffle the resampled data
df_upsampled_race_sex_max = df_upsampled_race_sex_max.sample(frac=1).reset_index(drop=True)

# Data Preparation
X = df_upsampled_race_sex_max.drop(['two_year_recid'], axis=1)
y = df_upsampled_race_sex_max['two_year_recid']

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Remove sex and race from the training data
X_train = X_train.drop(['sex', 'race'], axis=1)
sensitive_feature_race = X_test['race']
sensitive_feature_sex = X_test['sex']
X_test = X_test.drop(['sex', 'race'], axis=1)

# Initialize the Gradient Boosting classifier
gb_classifier = GradientBoostingClassifier(learning_rate=0.01, 
                                           max_depth=3, 
                                           n_estimators=300, 
                                           random_state=42)

# Fit the grid search to the data
gb_classifier.fit(X_train, y_train)

# Evaluate the model on the test set
y_pred_gb = gb_classifier.predict(X_test)

model_accuracy_gb = accuracy_score(y_test, y_pred_gb)
AUC = roc_auc_score(y_test, y_pred_gb)
f1 = f1_score(y_test, y_pred_gb)
precision = precision_score(y_test, y_pred_gb)
recall = recall_score(y_test, y_pred_gb)

statistical_parity_race = statistical_parity(y_test, y_pred_gb, sensitive_feature_race)
equalized_odds_race = equalized_odds(y_test, y_pred_gb, sensitive_feature_race)

statistical_parity_sex = statistical_parity(y_test, y_pred_gb, sensitive_feature_sex)
equalized_odds_sex = equalized_odds(y_test, y_pred_gb, sensitive_feature_sex)

resampling_results_df = update_results(
    resampling_group='Downsampling_Race_Upsampling_Sex',
    model_accuracy=model_accuracy_gb,  
    AUC=AUC,  
    f1=f1, 
    precision=precision,  
    recall=recall, 
    statistical_parity_sex =statistical_parity_sex, 
    equalized_odds_sex =equalized_odds_sex,
    statistical_parity_race = statistical_parity_race, 
    equalized_odds_race = equalized_odds_race
)

/var/folders/h5/r3ldzs6x5bxb71r4t20ydphh0000gn/T/ipykernel_98044/774575340.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return resampling_results_df.append(new_entry, ignore_index=True)


In [10]:
resampling_results_df

,resampling_group,model_accuracy,AUC,f1,precision,recall,statistical_parity_sex,equalized_odds_sex,statistical_parity_race,equalized_odds_race
0,Upsampled_Race_Sex,0.668972,0.668935,0.647002,0.692519,0.607100,"{'Statistical Parity Difference': 0.073, 'Stat...","{'Equalized Odds Difference': 0.073, 'Equalize...","{'Statistical Parity Difference': 0.138, 'Stat...","{'Equalized Odds Difference': 0.153, 'Equalize..."
1,Downsampled_Race_Sex,0.672515,0.674180,0.621622,0.747967,0.531792,"{'Statistical Parity Difference': 0.143, 'Stat...","{'Equalized Odds Difference': 0.202, 'Equalize...","{'Statistical Parity Difference': 0.053, 'Stat...","{'Equalized Odds Difference': 0.152, 'Equalize..."
2,Upsampling_Race_Downsampling_Sex,0.649547,0.647168,0.619672,0.630000,0.609677,"{'Statistical Parity Difference': 0.069, 'Stat...","{'Equalized Odds Difference': 0.109, 'Equalize...","{'Statistical Parity Difference': 0.196, 'Stat...","{'Equalized Odds Difference': 0.19, 'Equalized..."
3,Downsampling_Race_Upsampling_Sex,0.635193,0.634808,0.611365,0.649098,0.577778,"{'Statistical Parity Difference': 0.103, 'Stat...","{'Equalized Odds Difference': 0.109, 'Equalize...","{'Statistical Parity Difference': 0.14, 'Stati...","{'Equalized Odds Difference': 0.192, 'Equalize..."


## 5. Downsampling_Sex_Upsampling_Race

In [11]:
# Downsampling_Sex_Upsampling_Race
group_1 = df[df['sex'] == 1]
group_0 = df[df['sex'] == 0]


# Function to resample each subgroup to have the same number of samples
def resample_group(df, group_size):
    return df.sample(n=group_size, replace=True, random_state=42)

max_group_size = min(len(group_0[group_0['two_year_recid'] == 1]), len(group_1[group_1['two_year_recid'] == 1]))

# Resample each group to have the same number of samples
group_0_recidivate = resample_group(group_0[group_0['two_year_recid'] == 1], max_group_size)
group_1_recidivate = resample_group(group_1[group_1['two_year_recid'] == 1], max_group_size)

group_0_not_recidivate = resample_group(group_0[group_0['two_year_recid'] == 0], max_group_size)
group_1_not_recidivate = resample_group(group_1[group_1['two_year_recid'] == 0], max_group_size)

# Combine the resampled data back
df_upsampled_race = pd.concat([group_0_recidivate, group_0_not_recidivate, group_1_recidivate, group_1_not_recidivate])

# Shuffle the resampled data
df_upsampled_race = df_upsampled_race.sample(frac=1).reset_index(drop=True)

# Upsampling SEX feature
group_1 = df_upsampled_race[df_upsampled_race['race'] == 1]
group_0 = df_upsampled_race[df_upsampled_race['race'] == 0]

# Function to resample each subgroup to have the same number of samples
def resample_group(df, group_size):
    return df.sample(n=group_size, replace=True, random_state=42)

max_group_size = max(len(group_0[group_0['two_year_recid'] == 1]), len(group_1[group_1['two_year_recid'] == 1]))

# Resample each group to have the same number of samples
group_0_recidivate = resample_group(group_0[group_0['two_year_recid'] == 1], max_group_size)
group_1_recidivate = resample_group(group_1[group_1['two_year_recid'] == 1], max_group_size)

group_0_not_recidivate = resample_group(group_0[group_0['two_year_recid'] == 0], max_group_size)
group_1_not_recidivate = resample_group(group_1[group_1['two_year_recid'] == 0], max_group_size)

# Combine the resampled data back
df_upsampled_race_sex_max = pd.concat([group_0_recidivate, group_0_not_recidivate, group_1_recidivate, group_1_not_recidivate])

# Shuffle the resampled data
df_upsampled_race_sex_max = df_upsampled_race_sex_max.sample(frac=1).reset_index(drop=True)

# Data Preparation
X = df_upsampled_race_sex_max.drop(['two_year_recid'], axis=1)
y = df_upsampled_race_sex_max['two_year_recid']

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Remove sex and race from the training data
X_train = X_train.drop(['sex', 'race'], axis=1)
sensitive_feature_race = X_test['race']
sensitive_feature_sex = X_test['sex']
X_test = X_test.drop(['sex', 'race'], axis=1)

# Initialize the Gradient Boosting classifier
gb_classifier = GradientBoostingClassifier(learning_rate=0.01, 
                                           max_depth=3, 
                                           n_estimators=300, 
                                           random_state=42)

# Fit the grid search to the data
gb_classifier.fit(X_train, y_train)

# Evaluate the model on the test set
y_pred_gb = gb_classifier.predict(X_test)

model_accuracy_gb = accuracy_score(y_test, y_pred_gb)
AUC = roc_auc_score(y_test, y_pred_gb)
f1 = f1_score(y_test, y_pred_gb)
precision = precision_score(y_test, y_pred_gb)
recall = recall_score(y_test, y_pred_gb)

statistical_parity_race = statistical_parity(y_test, y_pred_gb, sensitive_feature_race)
equalized_odds_race = equalized_odds(y_test, y_pred_gb, sensitive_feature_race)

statistical_parity_sex = statistical_parity(y_test, y_pred_gb, sensitive_feature_sex)
equalized_odds_sex = equalized_odds(y_test, y_pred_gb, sensitive_feature_sex)

resampling_results_df = update_results(
    resampling_group='Downsampling_Sex_Upsampling_Race',
    model_accuracy=model_accuracy_gb,  
    AUC=AUC,  
    f1=f1, 
    precision=precision,  
    recall=recall, 
    statistical_parity_sex =statistical_parity_sex, 
    equalized_odds_sex =equalized_odds_sex,
    statistical_parity_race = statistical_parity_race, 
    equalized_odds_race = equalized_odds_race
)

/var/folders/h5/r3ldzs6x5bxb71r4t20ydphh0000gn/T/ipykernel_98044/774575340.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return resampling_results_df.append(new_entry, ignore_index=True)


In [12]:
resampling_results_df

,resampling_group,model_accuracy,AUC,f1,precision,recall,statistical_parity_sex,equalized_odds_sex,statistical_parity_race,equalized_odds_race
0,Upsampled_Race_Sex,0.668972,0.668935,0.647002,0.692519,0.607100,"{'Statistical Parity Difference': 0.073, 'Stat...","{'Equalized Odds Difference': 0.073, 'Equalize...","{'Statistical Parity Difference': 0.138, 'Stat...","{'Equalized Odds Difference': 0.153, 'Equalize..."
1,Downsampled_Race_Sex,0.672515,0.674180,0.621622,0.747967,0.531792,"{'Statistical Parity Difference': 0.143, 'Stat...","{'Equalized Odds Difference': 0.202, 'Equalize...","{'Statistical Parity Difference': 0.053, 'Stat...","{'Equalized Odds Difference': 0.152, 'Equalize..."
2,Upsampling_Race_Downsampling_Sex,0.649547,0.647168,0.619672,0.630000,0.609677,"{'Statistical Parity Difference': 0.069, 'Stat...","{'Equalized Odds Difference': 0.109, 'Equalize...","{'Statistical Parity Difference': 0.196, 'Stat...","{'Equalized Odds Difference': 0.19, 'Equalized..."
3,Downsampling_Race_Upsampling_Sex,0.635193,0.634808,0.611365,0.649098,0.577778,"{'Statistical Parity Difference': 0.103, 'Stat...","{'Equalized Odds Difference': 0.109, 'Equalize...","{'Statistical Parity Difference': 0.14, 'Stati...","{'Equalized Odds Difference': 0.192, 'Equalize..."
4,Downsampling_Sex_Upsampling_Race,0.668571,0.666993,0.641975,0.675325,0.611765,"{'Statistical Parity Difference': 0.085, 'Stat...","{'Equalized Odds Difference': 0.081, 'Equalize...","{'Statistical Parity Difference': 0.245, 'Stat...","{'Equalized Odds Difference': 0.356, 'Equalize..."


## 6. Upsampling_Sex_Downsampling_Race

In [13]:
# Upsampling_Sex_Downsampling_Race
group_1 = df[df['sex'] == 1]
group_0 = df[df['sex'] == 0]


# Function to resample each subgroup to have the same number of samples
def resample_group(df, group_size):
    return df.sample(n=group_size, replace=True, random_state=42)

max_group_size = max(len(group_0[group_0['two_year_recid'] == 1]), len(group_1[group_1['two_year_recid'] == 1]))

# Resample each group to have the same number of samples
group_0_recidivate = resample_group(group_0[group_0['two_year_recid'] == 1], max_group_size)
group_1_recidivate = resample_group(group_1[group_1['two_year_recid'] == 1], max_group_size)

group_0_not_recidivate = resample_group(group_0[group_0['two_year_recid'] == 0], max_group_size)
group_1_not_recidivate = resample_group(group_1[group_1['two_year_recid'] == 0], max_group_size)

# Combine the resampled data back
df_upsampled_race = pd.concat([group_0_recidivate, group_0_not_recidivate, group_1_recidivate, group_1_not_recidivate])

# Shuffle the resampled data
df_upsampled_race = df_upsampled_race.sample(frac=1).reset_index(drop=True)

# Upsampling SEX feature
group_1 = df_upsampled_race[df_upsampled_race['race'] == 1]
group_0 = df_upsampled_race[df_upsampled_race['race'] == 0]

# Function to resample each subgroup to have the same number of samples
def resample_group(df, group_size):
    return df.sample(n=group_size, replace=True, random_state=42)

max_group_size = min(len(group_0[group_0['two_year_recid'] == 1]), len(group_1[group_1['two_year_recid'] == 1]))

# Resample each group to have the same number of samples
group_0_recidivate = resample_group(group_0[group_0['two_year_recid'] == 1], max_group_size)
group_1_recidivate = resample_group(group_1[group_1['two_year_recid'] == 1], max_group_size)

group_0_not_recidivate = resample_group(group_0[group_0['two_year_recid'] == 0], max_group_size)
group_1_not_recidivate = resample_group(group_1[group_1['two_year_recid'] == 0], max_group_size)

# Combine the resampled data back
df_upsampled_race_sex_max = pd.concat([group_0_recidivate, group_0_not_recidivate, group_1_recidivate, group_1_not_recidivate])

# Shuffle the resampled data
df_upsampled_race_sex_max = df_upsampled_race_sex_max.sample(frac=1).reset_index(drop=True)

# Data Preparation
X = df_upsampled_race_sex_max.drop(['two_year_recid'], axis=1)
y = df_upsampled_race_sex_max['two_year_recid']

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Remove sex and race from the training data
X_train = X_train.drop(['sex', 'race'], axis=1)
sensitive_feature_race = X_test['race']
sensitive_feature_sex = X_test['sex']
X_test = X_test.drop(['sex', 'race'], axis=1)

# Initialize the Gradient Boosting classifier
gb_classifier = GradientBoostingClassifier(learning_rate=0.01, 
                                           max_depth=3, 
                                           n_estimators=300, 
                                           random_state=42)

# Fit the grid search to the data
gb_classifier.fit(X_train, y_train)

# Evaluate the model on the test set
y_pred_gb = gb_classifier.predict(X_test)

model_accuracy_gb = accuracy_score(y_test, y_pred_gb)
AUC = roc_auc_score(y_test, y_pred_gb)
f1 = f1_score(y_test, y_pred_gb)
precision = precision_score(y_test, y_pred_gb)
recall = recall_score(y_test, y_pred_gb)

statistical_parity_race = statistical_parity(y_test, y_pred_gb, sensitive_feature_race)
equalized_odds_race = equalized_odds(y_test, y_pred_gb, sensitive_feature_race)

statistical_parity_sex = statistical_parity(y_test, y_pred_gb, sensitive_feature_sex)
equalized_odds_sex = equalized_odds(y_test, y_pred_gb, sensitive_feature_sex)



In [14]:
resampling_results_df = update_results(
    resampling_group='Upsampling_Sex_Downsampling_Race',
    model_accuracy=model_accuracy_gb,  
    AUC=AUC,  
    f1=f1, 
    precision=precision,  
    recall=recall, 
    statistical_parity_sex =statistical_parity_sex, 
    equalized_odds_sex =equalized_odds_sex,
    statistical_parity_race = statistical_parity_race, 
    equalized_odds_race = equalized_odds_race
)

/var/folders/h5/r3ldzs6x5bxb71r4t20ydphh0000gn/T/ipykernel_98044/774575340.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return resampling_results_df.append(new_entry, ignore_index=True)


In [15]:
resampling_results_df

,resampling_group,model_accuracy,AUC,f1,precision,recall,statistical_parity_sex,equalized_odds_sex,statistical_parity_race,equalized_odds_race
0,Upsampled_Race_Sex,0.668972,0.668935,0.647002,0.692519,0.607100,"{'Statistical Parity Difference': 0.073, 'Stat...","{'Equalized Odds Difference': 0.073, 'Equalize...","{'Statistical Parity Difference': 0.138, 'Stat...","{'Equalized Odds Difference': 0.153, 'Equalize..."
1,Downsampled_Race_Sex,0.672515,0.674180,0.621622,0.747967,0.531792,"{'Statistical Parity Difference': 0.143, 'Stat...","{'Equalized Odds Difference': 0.202, 'Equalize...","{'Statistical Parity Difference': 0.053, 'Stat...","{'Equalized Odds Difference': 0.152, 'Equalize..."
2,Upsampling_Race_Downsampling_Sex,0.649547,0.647168,0.619672,0.630000,0.609677,"{'Statistical Parity Difference': 0.069, 'Stat...","{'Equalized Odds Difference': 0.109, 'Equalize...","{'Statistical Parity Difference': 0.196, 'Stat...","{'Equalized Odds Difference': 0.19, 'Equalized..."
3,Downsampling_Race_Upsampling_Sex,0.635193,0.634808,0.611365,0.649098,0.577778,"{'Statistical Parity Difference': 0.103, 'Stat...","{'Equalized Odds Difference': 0.109, 'Equalize...","{'Statistical Parity Difference': 0.14, 'Stati...","{'Equalized Odds Difference': 0.192, 'Equalize..."
4,Downsampling_Sex_Upsampling_Race,0.668571,0.666993,0.641975,0.675325,0.611765,"{'Statistical Parity Difference': 0.085, 'Stat...","{'Equalized Odds Difference': 0.081, 'Equalize...","{'Statistical Parity Difference': 0.245, 'Stat...","{'Equalized Odds Difference': 0.356, 'Equalize..."
5,Upsampling_Sex_Downsampling_Race,0.636643,0.639698,0.603352,0.688776,0.536779,"{'Statistical Parity Difference': 0.133, 'Stat...","{'Equalized Odds Difference': 0.134, 'Equalize...","{'Statistical Parity Difference': 0.164, 'Stat...","{'Equalized Odds Difference': 0.22, 'Equalized..."


## 7. Upsampling_Sex_Race


In [16]:
# Upsampling_Sex_Race
group_1 = df[df['sex'] == 1]
group_0 = df[df['sex'] == 0]


# Function to resample each subgroup to have the same number of samples
def resample_group(df, group_size):
    return df.sample(n=group_size, replace=True, random_state=42)

max_group_size = max(len(group_0[group_0['two_year_recid'] == 1]), len(group_1[group_1['two_year_recid'] == 1]))

# Resample each group to have the same number of samples
group_0_recidivate = resample_group(group_0[group_0['two_year_recid'] == 1], max_group_size)
group_1_recidivate = resample_group(group_1[group_1['two_year_recid'] == 1], max_group_size)

group_0_not_recidivate = resample_group(group_0[group_0['two_year_recid'] == 0], max_group_size)
group_1_not_recidivate = resample_group(group_1[group_1['two_year_recid'] == 0], max_group_size)

# Combine the resampled data back
df_upsampled_race = pd.concat([group_0_recidivate, group_0_not_recidivate, group_1_recidivate, group_1_not_recidivate])

# Shuffle the resampled data
df_upsampled_race = df_upsampled_race.sample(frac=1).reset_index(drop=True)

# Upsampling SEX feature
group_1 = df_upsampled_race[df_upsampled_race['race'] == 1]
group_0 = df_upsampled_race[df_upsampled_race['race'] == 0]

# Function to resample each subgroup to have the same number of samples
def resample_group(df, group_size):
    return df.sample(n=group_size, replace=True, random_state=42)

max_group_size = max(len(group_0[group_0['two_year_recid'] == 1]), len(group_1[group_1['two_year_recid'] == 1]))

# Resample each group to have the same number of samples
group_0_recidivate = resample_group(group_0[group_0['two_year_recid'] == 1], max_group_size)
group_1_recidivate = resample_group(group_1[group_1['two_year_recid'] == 1], max_group_size)

group_0_not_recidivate = resample_group(group_0[group_0['two_year_recid'] == 0], max_group_size)
group_1_not_recidivate = resample_group(group_1[group_1['two_year_recid'] == 0], max_group_size)

# Combine the resampled data back
df_upsampled_race_sex_max = pd.concat([group_0_recidivate, group_0_not_recidivate, group_1_recidivate, group_1_not_recidivate])

# Shuffle the resampled data
df_upsampled_race_sex_max = df_upsampled_race_sex_max.sample(frac=1).reset_index(drop=True)

# Data Preparation
X = df_upsampled_race_sex_max.drop(['two_year_recid'], axis=1)
y = df_upsampled_race_sex_max['two_year_recid']

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Remove sex and race from the training data
X_train = X_train.drop(['sex', 'race'], axis=1)
sensitive_feature_race = X_test['race']
sensitive_feature_sex = X_test['sex']
X_test = X_test.drop(['sex', 'race'], axis=1)

# Initialize the Gradient Boosting classifier
gb_classifier = GradientBoostingClassifier(learning_rate=0.01, 
                                           max_depth=3, 
                                           n_estimators=300, 
                                           random_state=42)

# Fit the grid search to the data
gb_classifier.fit(X_train, y_train)

# Evaluate the model on the test set
y_pred_gb = gb_classifier.predict(X_test)

model_accuracy_gb = accuracy_score(y_test, y_pred_gb)
AUC = roc_auc_score(y_test, y_pred_gb)
f1 = f1_score(y_test, y_pred_gb)
precision = precision_score(y_test, y_pred_gb)
recall = recall_score(y_test, y_pred_gb)

statistical_parity_race = statistical_parity(y_test, y_pred_gb, sensitive_feature_race)
equalized_odds_race = equalized_odds(y_test, y_pred_gb, sensitive_feature_race)

statistical_parity_sex = statistical_parity(y_test, y_pred_gb, sensitive_feature_sex)
equalized_odds_sex = equalized_odds(y_test, y_pred_gb, sensitive_feature_sex)

resampling_results_df = update_results(
    resampling_group='Upsampling_Sex_Race',
    model_accuracy=model_accuracy_gb,  
    AUC=AUC,  
    f1=f1, 
    precision=precision,  
    recall=recall, 
    statistical_parity_sex =statistical_parity_sex, 
    equalized_odds_sex =equalized_odds_sex,
    statistical_parity_race = statistical_parity_race, 
    equalized_odds_race = equalized_odds_race
)

/var/folders/h5/r3ldzs6x5bxb71r4t20ydphh0000gn/T/ipykernel_98044/774575340.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return resampling_results_df.append(new_entry, ignore_index=True)


In [17]:
resampling_results_df

,resampling_group,model_accuracy,AUC,f1,precision,recall,statistical_parity_sex,equalized_odds_sex,statistical_parity_race,equalized_odds_race
0,Upsampled_Race_Sex,0.668972,0.668935,0.647002,0.692519,0.607100,"{'Statistical Parity Difference': 0.073, 'Stat...","{'Equalized Odds Difference': 0.073, 'Equalize...","{'Statistical Parity Difference': 0.138, 'Stat...","{'Equalized Odds Difference': 0.153, 'Equalize..."
1,Downsampled_Race_Sex,0.672515,0.674180,0.621622,0.747967,0.531792,"{'Statistical Parity Difference': 0.143, 'Stat...","{'Equalized Odds Difference': 0.202, 'Equalize...","{'Statistical Parity Difference': 0.053, 'Stat...","{'Equalized Odds Difference': 0.152, 'Equalize..."
2,Upsampling_Race_Downsampling_Sex,0.649547,0.647168,0.619672,0.630000,0.609677,"{'Statistical Parity Difference': 0.069, 'Stat...","{'Equalized Odds Difference': 0.109, 'Equalize...","{'Statistical Parity Difference': 0.196, 'Stat...","{'Equalized Odds Difference': 0.19, 'Equalized..."
3,Downsampling_Race_Upsampling_Sex,0.635193,0.634808,0.611365,0.649098,0.577778,"{'Statistical Parity Difference': 0.103, 'Stat...","{'Equalized Odds Difference': 0.109, 'Equalize...","{'Statistical Parity Difference': 0.14, 'Stati...","{'Equalized Odds Difference': 0.192, 'Equalize..."
4,Downsampling_Sex_Upsampling_Race,0.668571,0.666993,0.641975,0.675325,0.611765,"{'Statistical Parity Difference': 0.085, 'Stat...","{'Equalized Odds Difference': 0.081, 'Equalize...","{'Statistical Parity Difference': 0.245, 'Stat...","{'Equalized Odds Difference': 0.356, 'Equalize..."
5,Upsampling_Sex_Downsampling_Race,0.636643,0.639698,0.603352,0.688776,0.536779,"{'Statistical Parity Difference': 0.133, 'Stat...","{'Equalized Odds Difference': 0.134, 'Equalize...","{'Statistical Parity Difference': 0.164, 'Stat...","{'Equalized Odds Difference': 0.22, 'Equalized..."
6,Upsampling_Sex_Race,0.647059,0.647207,0.611740,0.681102,0.555199,"{'Statistical Parity Difference': 0.109, 'Stat...","{'Equalized Odds Difference': 0.124, 'Equalize...","{'Statistical Parity Difference': 0.164, 'Stat...","{'Equalized Odds Difference': 0.22, 'Equalized..."


## 8. Downsampling_Sex_Race

In [18]:
# Downsampling_Sex_Race
group_1 = df[df['sex'] == 1]
group_0 = df[df['sex'] == 0]


# Function to resample each subgroup to have the same number of samples
def resample_group(df, group_size):
    return df.sample(n=group_size, replace=True, random_state=42)

max_group_size = min(len(group_0[group_0['two_year_recid'] == 1]), len(group_1[group_1['two_year_recid'] == 1]))

# Resample each group to have the same number of samples
group_0_recidivate = resample_group(group_0[group_0['two_year_recid'] == 1], max_group_size)
group_1_recidivate = resample_group(group_1[group_1['two_year_recid'] == 1], max_group_size)

group_0_not_recidivate = resample_group(group_0[group_0['two_year_recid'] == 0], max_group_size)
group_1_not_recidivate = resample_group(group_1[group_1['two_year_recid'] == 0], max_group_size)

# Combine the resampled data back
df_upsampled_race = pd.concat([group_0_recidivate, group_0_not_recidivate, group_1_recidivate, group_1_not_recidivate])

# Shuffle the resampled data
df_upsampled_race = df_upsampled_race.sample(frac=1).reset_index(drop=True)

# Upsampling SEX feature
group_1 = df_upsampled_race[df_upsampled_race['race'] == 1]
group_0 = df_upsampled_race[df_upsampled_race['race'] == 0]

# Function to resample each subgroup to have the same number of samples
def resample_group(df, group_size):
    return df.sample(n=group_size, replace=True, random_state=42)

max_group_size = min(len(group_0[group_0['two_year_recid'] == 1]), len(group_1[group_1['two_year_recid'] == 1]))

# Resample each group to have the same number of samples
group_0_recidivate = resample_group(group_0[group_0['two_year_recid'] == 1], max_group_size)
group_1_recidivate = resample_group(group_1[group_1['two_year_recid'] == 1], max_group_size)

group_0_not_recidivate = resample_group(group_0[group_0['two_year_recid'] == 0], max_group_size)
group_1_not_recidivate = resample_group(group_1[group_1['two_year_recid'] == 0], max_group_size)

# Combine the resampled data back
df_upsampled_race_sex_max = pd.concat([group_0_recidivate, group_0_not_recidivate, group_1_recidivate, group_1_not_recidivate])

# Shuffle the resampled data
df_upsampled_race_sex_max = df_upsampled_race_sex_max.sample(frac=1).reset_index(drop=True)

# Data Preparation
X = df_upsampled_race_sex_max.drop(['two_year_recid'], axis=1)
y = df_upsampled_race_sex_max['two_year_recid']

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Remove sex and race from the training data
X_train = X_train.drop(['sex', 'race'], axis=1)
sensitive_feature_race = X_test['race']
sensitive_feature_sex = X_test['sex']
X_test = X_test.drop(['sex', 'race'], axis=1)

# Initialize the Gradient Boosting classifier
gb_classifier = GradientBoostingClassifier(learning_rate=0.01, 
                                           max_depth=3, 
                                           n_estimators=300, 
                                           random_state=42)

# Fit the grid search to the data
gb_classifier.fit(X_train, y_train)

# Evaluate the model on the test set
y_pred_gb = gb_classifier.predict(X_test)

model_accuracy_gb = accuracy_score(y_test, y_pred_gb)
AUC = roc_auc_score(y_test, y_pred_gb)
f1 = f1_score(y_test, y_pred_gb)
precision = precision_score(y_test, y_pred_gb)
recall = recall_score(y_test, y_pred_gb)

statistical_parity_race = statistical_parity(y_test, y_pred_gb, sensitive_feature_race)
equalized_odds_race = equalized_odds(y_test, y_pred_gb, sensitive_feature_race)

statistical_parity_sex = statistical_parity(y_test, y_pred_gb, sensitive_feature_sex)
equalized_odds_sex = equalized_odds(y_test, y_pred_gb, sensitive_feature_sex)

resampling_results_df = update_results(
    resampling_group='Downsampling_Sex_Race',
    model_accuracy=model_accuracy_gb,  
    AUC=AUC,  
    f1=f1, 
    precision=precision,  
    recall=recall, 
    statistical_parity_sex =statistical_parity_sex, 
    equalized_odds_sex =equalized_odds_sex,
    statistical_parity_race = statistical_parity_race, 
    equalized_odds_race = equalized_odds_race
)

/var/folders/h5/r3ldzs6x5bxb71r4t20ydphh0000gn/T/ipykernel_98044/774575340.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return resampling_results_df.append(new_entry, ignore_index=True)


In [19]:
resampling_results_df

,resampling_group,model_accuracy,AUC,f1,precision,recall,statistical_parity_sex,equalized_odds_sex,statistical_parity_race,equalized_odds_race
0,Upsampled_Race_Sex,0.668972,0.668935,0.647002,0.692519,0.607100,"{'Statistical Parity Difference': 0.073, 'Stat...","{'Equalized Odds Difference': 0.073, 'Equalize...","{'Statistical Parity Difference': 0.138, 'Stat...","{'Equalized Odds Difference': 0.153, 'Equalize..."
1,Downsampled_Race_Sex,0.672515,0.674180,0.621622,0.747967,0.531792,"{'Statistical Parity Difference': 0.143, 'Stat...","{'Equalized Odds Difference': 0.202, 'Equalize...","{'Statistical Parity Difference': 0.053, 'Stat...","{'Equalized Odds Difference': 0.152, 'Equalize..."
2,Upsampling_Race_Downsampling_Sex,0.649547,0.647168,0.619672,0.630000,0.609677,"{'Statistical Parity Difference': 0.069, 'Stat...","{'Equalized Odds Difference': 0.109, 'Equalize...","{'Statistical Parity Difference': 0.196, 'Stat...","{'Equalized Odds Difference': 0.19, 'Equalized..."
3,Downsampling_Race_Upsampling_Sex,0.635193,0.634808,0.611365,0.649098,0.577778,"{'Statistical Parity Difference': 0.103, 'Stat...","{'Equalized Odds Difference': 0.109, 'Equalize...","{'Statistical Parity Difference': 0.14, 'Stati...","{'Equalized Odds Difference': 0.192, 'Equalize..."
4,Downsampling_Sex_Upsampling_Race,0.668571,0.666993,0.641975,0.675325,0.611765,"{'Statistical Parity Difference': 0.085, 'Stat...","{'Equalized Odds Difference': 0.081, 'Equalize...","{'Statistical Parity Difference': 0.245, 'Stat...","{'Equalized Odds Difference': 0.356, 'Equalize..."
5,Upsampling_Sex_Downsampling_Race,0.636643,0.639698,0.603352,0.688776,0.536779,"{'Statistical Parity Difference': 0.133, 'Stat...","{'Equalized Odds Difference': 0.134, 'Equalize...","{'Statistical Parity Difference': 0.164, 'Stat...","{'Equalized Odds Difference': 0.22, 'Equalized..."
6,Upsampling_Sex_Race,0.647059,0.647207,0.611740,0.681102,0.555199,"{'Statistical Parity Difference': 0.109, 'Stat...","{'Equalized Odds Difference': 0.124, 'Equalize...","{'Statistical Parity Difference': 0.164, 'Stat...","{'Equalized Odds Difference': 0.22, 'Equalized..."
7,Downsampling_Sex_Race,0.609164,0.610670,0.600551,0.637427,0.567708,"{'Statistical Parity Difference': 0.024, 'Stat...","{'Equalized Odds Difference': 0.055, 'Equalize...","{'Statistical Parity Difference': 0.247, 'Stat...","{'Equalized Odds Difference': 0.279, 'Equalize..."


# Resampling analysis

In [20]:
resampling_analysis= pd.DataFrame()

In [21]:
# Correcting the key bnames and extracting the values for the new DataFrame
resampling_analysis['resampling_group'] = resampling_results_df['resampling_group']
resampling_analysis['statistical_parity_difference_sex'] = resampling_results_df['statistical_parity_sex'].apply(lambda x: x['Statistical Parity Difference'])
resampling_analysis['statistical_parity_difference_race'] = resampling_results_df['statistical_parity_race'].apply(lambda x: x['Statistical Parity Difference'])
resampling_analysis['equalized_odds_difference_sex'] = resampling_results_df['equalized_odds_sex'].apply(lambda x: x['Equalized Odds Difference'])
resampling_analysis['equalized_odds_difference_race'] = resampling_results_df['equalized_odds_race'].apply(lambda x: x['Equalized Odds Difference'])
resampling_analysis['statistical_parity_ratio_sex'] = resampling_results_df['statistical_parity_sex'].apply(lambda x: x['Statistical Parity ratio'])
resampling_analysis['statistical_parity_ratio_race'] = resampling_results_df['statistical_parity_race'].apply(lambda x: x['Statistical Parity ratio'])
resampling_analysis['equalized_odds_ratio_sex'] = resampling_results_df['equalized_odds_sex'].apply(lambda x: x['Equalized Odds Ratio'])
resampling_analysis['equalized_odds_ratio_race'] = resampling_results_df['equalized_odds_race'].apply(lambda x: x['Equalized Odds Ratio'])


In [22]:
resampling_analysis

,resampling_group,statistical_parity_difference_sex,statistical_parity_difference_race,equalized_odds_difference_sex,equalized_odds_difference_race,statistical_parity_ratio_sex,statistical_parity_ratio_race,equalized_odds_ratio_sex,equalized_odds_ratio_race
0,Upsampled_Race_Sex,0.073,0.138,0.073,0.153,0.847,0.729,0.787,0.616
1,Downsampled_Race_Sex,0.143,0.053,0.202,0.152,0.672,0.864,0.305,0.414
2,Upsampling_Race_Downsampling_Sex,0.069,0.196,0.109,0.190,0.859,0.648,0.835,0.550
3,Downsampling_Race_Upsampling_Sex,0.103,0.140,0.109,0.192,0.791,0.729,0.698,0.677
4,Downsampling_Sex_Upsampling_Race,0.085,0.245,0.081,0.356,0.825,0.563,0.809,0.549
5,Upsampling_Sex_Downsampling_Race,0.133,0.164,0.134,0.220,0.716,0.661,0.601,0.604
6,Upsampling_Sex_Race,0.109,0.164,0.124,0.220,0.764,0.666,0.640,0.635
7,Downsampling_Sex_Race,0.024,0.247,0.055,0.279,0.950,0.581,0.853,0.423


In [23]:
# Adding new columns to sum up the differences and ratios row-wise
resampling_analysis['sum_differences'] = resampling_analysis['statistical_parity_difference_sex'] + resampling_analysis['statistical_parity_difference_race'] + \
                            resampling_analysis['equalized_odds_difference_sex'] + resampling_analysis['equalized_odds_difference_race']

resampling_analysis['sum_ratios'] = resampling_analysis['statistical_parity_ratio_sex'] + resampling_analysis['statistical_parity_ratio_race'] + \
                       resampling_analysis['equalized_odds_ratio_sex'] + resampling_analysis['equalized_odds_ratio_race']

resampling_analysis


,resampling_group,statistical_parity_difference_sex,statistical_parity_difference_race,equalized_odds_difference_sex,equalized_odds_difference_race,statistical_parity_ratio_sex,statistical_parity_ratio_race,equalized_odds_ratio_sex,equalized_odds_ratio_race,sum_differences,sum_ratios
0,Upsampled_Race_Sex,0.073,0.138,0.073,0.153,0.847,0.729,0.787,0.616,0.437,2.979
1,Downsampled_Race_Sex,0.143,0.053,0.202,0.152,0.672,0.864,0.305,0.414,0.550,2.255
2,Upsampling_Race_Downsampling_Sex,0.069,0.196,0.109,0.190,0.859,0.648,0.835,0.550,0.564,2.892
3,Downsampling_Race_Upsampling_Sex,0.103,0.140,0.109,0.192,0.791,0.729,0.698,0.677,0.544,2.895
4,Downsampling_Sex_Upsampling_Race,0.085,0.245,0.081,0.356,0.825,0.563,0.809,0.549,0.767,2.746
5,Upsampling_Sex_Downsampling_Race,0.133,0.164,0.134,0.220,0.716,0.661,0.601,0.604,0.651,2.582
6,Upsampling_Sex_Race,0.109,0.164,0.124,0.220,0.764,0.666,0.640,0.635,0.617,2.705
7,Downsampling_Sex_Race,0.024,0.247,0.055,0.279,0.950,0.581,0.853,0.423,0.605,2.807


In [24]:
# Finding the row with the lowest sum of differences
row_lowest_sum_diff = resampling_analysis.loc[resampling_analysis['sum_differences'] == resampling_analysis['sum_differences'].min() ]

# Finding the row with the highest sum of ratios
row_highest_sum_ratio = resampling_analysis.loc[resampling_analysis['sum_ratios'] == resampling_analysis['sum_ratios'].max()]

row_lowest_sum_diff

,resampling_group,statistical_parity_difference_sex,statistical_parity_difference_race,equalized_odds_difference_sex,equalized_odds_difference_race,statistical_parity_ratio_sex,statistical_parity_ratio_race,equalized_odds_ratio_sex,equalized_odds_ratio_race,sum_differences,sum_ratios
0,Upsampled_Race_Sex,0.073,0.138,0.073,0.153,0.847,0.729,0.787,0.616,0.437,2.979


In [25]:
row_highest_sum_ratio

,resampling_group,statistical_parity_difference_sex,statistical_parity_difference_race,equalized_odds_difference_sex,equalized_odds_difference_race,statistical_parity_ratio_sex,statistical_parity_ratio_race,equalized_odds_ratio_sex,equalized_odds_ratio_race,sum_differences,sum_ratios
0,Upsampled_Race_Sex,0.073,0.138,0.073,0.153,0.847,0.729,0.787,0.616,0.437,2.979


In [27]:
resampling_results_df.to_csv('data/resampling_analysis_result.csv', index=False)